In [ ]:
#crtl f9 inicializar tudo

#Importações

***Bibliotecas e instalações***

In [ ]:
pip install pyspark

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

Importando funcções padrão. Função Window também importada porque será usado mais na frente

In [ ]:
import os
from google.cloud import storage

from pyspark.sql import SparkSession, Window
spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.functions import to_timestamp, to_date

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,BooleanType,DateType,TimestampType
import pandas as pd


# Criando Esquema e Dataframe

In [ ]:
customSchema = StructType([
   StructField("tempo_cargo_anterior", DoubleType(),True),
   StructField("id_usuario", IntegerType(),True),
   StructField("promocoes", IntegerType(),True),
   StructField("dias_cargo_anterior", IntegerType(),True),
   StructField("idade", IntegerType(),True),
   StructField("desfoque", DoubleType(),True),
   StructField("raiva", DoubleType(),True),
   StructField("desgosto", DoubleType(),True),
   StructField("medo", DoubleType(),True),
   StructField("felicidade", DoubleType(),True),
   StructField("neutro", DoubleType(),True),
   StructField("triste", DoubleType(),True),
   StructField("surpresa", DoubleType(),True),
   StructField("etnia", StringType(),True),
   StructField("genero", StringType(),True),
   StructField("oculos", StringType(),True),
   StructField("sorriso", DoubleType(),True),
   StructField("nacionalidade", StringType(),True),
   StructField("seguidores", IntegerType(),True),
   StructField("qualidade_imagem", DoubleType(),True)

])

schema = customSchema

In [ ]:

dfspark = pd.read_csv('gs://projetofinalgrupo8/saida/linkedin_tratado_pandas.csv', sep=',', header=0)
df= spark.createDataFrame(dfspark,schema=schema)

In [ ]:
df.show(20)

+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+
|tempo_cargo_anterior|id_usuario|promocoes|dias_cargo_anterior|idade|desfoque|raiva|desgosto|medo|felicidade|neutro|triste|surpresa|   etnia|   genero|      oculos|sorriso| nacionalidade|seguidores|qualidade_imagem|
+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+
|                 2.0|      3008|        1|                457|   37|    0.42| 76.6|     0.7| 0.7|     99.04|  0.09|  0.01|    0.09|Asiático|Masculino|Oculos_comum|  77.54|asiatico_leste|       420|           67.66|
|                 1.0|      3010|        1|               1827|   37|   64.26|  8.2|     8.2| 8.2|      9.59| 63.07|   0.1|    27.0|  Br

#Arredondando os valores das colunas necessárias


In [ ]:
df = (df.withColumn("tempo_cargo_anterior",F.round(F.col("tempo_cargo_anterior"),2))
        .withColumn("desfoque",F.round(F.col("desfoque"),2))
        .withColumn("raiva",F.round(F.col("raiva"),2))
        .withColumn("felicidade",F.round(F.col("felicidade"),2))
        .withColumn("neutro",F.round(F.col("neutro"),2))
        .withColumn("triste",F.round(F.col("triste"),2))
        .withColumn("surpresa",F.round(F.col("surpresa"),2))
        .withColumn("sorriso",F.round(F.col("sorriso"),2))
        .withColumn("qualidade_imagem",F.round(F.col("qualidade_imagem"),2))      
)


In [ ]:
df.show(5)

+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+
|tempo_cargo_anterior|id_usuario|promocoes|dias_cargo_anterior|idade|desfoque|raiva|desgosto|medo|felicidade|neutro|triste|surpresa|   etnia|   genero|      oculos|sorriso| nacionalidade|seguidores|qualidade_imagem|
+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+
|                 2.0|      3008|        1|                457|   37|    0.42| 76.6|     0.7| 0.7|     99.04|  0.09|  0.01|    0.09|Asiático|Masculino|Oculos_comum|  77.54|asiatico_leste|       420|           67.66|
|                 1.0|      3010|        1|               1827|   37|   64.26|  8.2|     8.2| 8.2|      9.59| 63.07|   0.1|    27.0|  Br

#Utilizando o comando 'GroupBy'

In [ ]:
df.groupBy("id_usuario").avg("tempo_cargo_anterior").show(10,truncate=False)

+----------+-------------------------+
|id_usuario|avg(tempo_cargo_anterior)|
+----------+-------------------------+
|3175      |1.0                      |
|3749      |1.0                      |
|3794      |1.0                      |
|3918      |1.0                      |
|3997      |2.0                      |
|4101      |1.0                      |
|4900      |1.0                      |
|4935      |1.0                      |
|5156      |1.0                      |
|5300      |1.0                      |
+----------+-------------------------+
only showing top 10 rows



***Agrupando a coluna "nacionalidade" e utilizando Aggregate para fazer pesquisas***

In [ ]:
(df.groupBy(F.col("nacionalidade")).agg(
            F.round(F.mean("promocoes"), 2).alias("media_promocoes"),
            F.round(F.max("promocoes"), 2).alias("max_promocoes"),
            F.round(F.mean("idade"), 2).alias("media_idade"),
            F.round(F.min("idade"), 2).alias("min_idade"),
            F.max("idade").alias("max_idade")
            ).orderBy("max_promocoes")
            .show())

+--------------+---------------+-------------+-----------+---------+---------+
| nacionalidade|media_promocoes|max_promocoes|media_idade|min_idade|max_idade|
+--------------+---------------+-------------+-----------+---------+---------+
|asiatico_leste|            1.0|            1|      34.37|       16|       75|
|        ingles|            1.0|            1|      47.14|       16|       80|
|     muculmano|            1.0|            1|      40.71|       19|       71|
|         judeu|            1.0|            1|      47.16|       26|       66|
|         grego|            1.0|            1|       44.6|       22|       69|
|       europeu|            1.0|            1|      44.73|       17|       73|
|     hispanico|            1.0|            1|      42.51|       16|       76|
|  asiatico_sul|            1.0|            1|      38.04|       16|       71|
|       nordico|            1.0|            1|      47.35|       26|       70|
|      africano|            1.0|            1|      


***Agrupando a coluna "idade" e utilizando Aggregate para fazer pesquisas***

In [ ]:
(df.groupBy(F.col("idade")).agg(
            F.round(F.max("qualidade_imagem"), 2).alias("max_qualidade"),
            F.round(F.mean("qualidade_imagem"), 2).alias("media_qualidade"),
            F.round(F.min("qualidade_imagem"), 2).alias("min_qualidade")
            ).orderBy("media_qualidade").show(20))

+-----+-------------+---------------+-------------+
|idade|max_qualidade|media_qualidade|min_qualidade|
+-----+-------------+---------------+-------------+
|   80|         0.01|           0.01|         0.01|
|   75|         0.01|           0.01|         0.01|
|   76|        88.55|          29.52|         0.01|
|   66|        93.04|          58.39|         0.01|
|   61|        94.28|          59.42|         0.01|
|   62|        94.43|          62.88|         0.01|
|   58|        94.48|          63.32|         0.01|
|   71|        92.44|          63.32|         0.01|
|   36|        94.66|          64.39|         0.01|
|   63|        94.49|          64.45|         0.01|
|   23|        94.49|          64.71|         0.01|
|   19|        94.01|          64.88|         0.01|
|   21|        94.32|          65.32|         0.01|
|   35|        94.68|           65.5|         0.01|
|   38|        94.79|          65.62|         0.01|
|   64|        94.66|          65.67|         0.01|
|   30|     

In [ ]:
(df.groupBy(F.col("idade")).agg(
            F.count("idade").alias("quantidade"),
            F.round(F.max("sorriso"), 2).alias("max_sorriso"),
            F.round(F.mean("sorriso"), 2).alias("media_sorriso"),
            F.round(F.min("sorriso"), 2).alias("min_sorriso")
            ).orderBy("idade").show(20))

+-----+----------+-----------+-------------+-----------+
|idade|quantidade|max_sorriso|media_sorriso|min_sorriso|
+-----+----------+-----------+-------------+-----------+
|   16|        18|      99.37|         74.7|       3.96|
|   17|        14|      99.58|        62.45|       3.54|
|   18|        23|      99.23|        65.08|        1.6|
|   19|        43|      99.48|        68.52|       2.01|
|   20|        55|      98.96|        63.91|       1.84|
|   21|        61|      99.19|        59.87|       1.12|
|   22|        78|      99.46|        50.66|       0.43|
|   23|        76|       99.0|         57.8|       1.21|
|   24|       107|      98.94|        62.05|       0.73|
|   25|       104|      99.67|        61.34|       0.54|
|   26|       147|      99.51|        55.28|       0.45|
|   27|       172|      99.62|        53.83|       0.54|
|   28|       174|      99.75|         57.8|       0.42|
|   29|       163|       99.4|        58.47|       0.92|
|   30|       216|      99.73| 


Agrupando a coluna "genero" e utilizando Aggregate para fazer pesquisas

In [ ]:
(df.groupBy(F.col("genero")).agg(
            F.round(F.mean("promocoes"), 2).alias("media_promocoes"),
            F.round(F.max("promocoes"), 2).alias("max_promocoes"),
            F.round(F.mean("idade"), 2).alias("media_idade"),
            F.round(F.min("idade"), 2).alias("min_idade"),
            F.max("idade").alias("maior_idade")
            ).orderBy("max_promocoes").show())

+---------+---------------+-------------+-----------+---------+-----------+
|   genero|media_promocoes|max_promocoes|media_idade|min_idade|maior_idade|
+---------+---------------+-------------+-----------+---------+-----------+
| Feminino|            1.0|            1|      42.55|       16|         80|
|Masculino|            1.0|            1|      43.23|       16|         76|
+---------+---------------+-------------+-----------+---------+-----------+



#Utilizando o comando 'Window'

***Odernando por idade e selecionando apenas algumas colunas***

In [ ]:
w0 = Window.partitionBy(F.col("idade")).orderBy("idade")

(df.withColumn("Rank",F.rank().over(w0))
  .withColumn("idade",F.max(F.col("idade")).over(w0))
  .select("id_usuario","genero","idade","nacionalidade","dias_cargo_anterior")
  .filter(F.col("dias_cargo_anterior") >1).show(10)
)

+----------+---------+-----+--------------+-------------------+
|id_usuario|   genero|idade| nacionalidade|dias_cargo_anterior|
+----------+---------+-----+--------------+-------------------+
|      3511|Masculino|   16|asiatico_leste|               1003|
|      3635|Masculino|   16|     hispanico|                366|
|      4451|Masculino|   16|     hispanico|                 61|
|      4902|Masculino|   16|        ingles|                152|
|      6235|Masculino|   16|asiatico_leste|               1219|
|      6990| Feminino|   16|asiatico_leste|                762|
|      7008| Feminino|   16|        ingles|                730|
|      7962| Feminino|   16|asiatico_leste|                 61|
|      8158| Feminino|   16|asiatico_leste|                640|
|      9054| Feminino|   16|     hispanico|               3012|
+----------+---------+-----+--------------+-------------------+
only showing top 10 rows



***Odernando por "nacionalidade" e selecionando apenas algumas colunas***

In [ ]:
w0 = Window.partitionBy(F.col("nacionalidade")).orderBy("idade")

(df.withColumn("Rank",F.rank().over(w0))
  .withColumn("idade",F.max(F.col("idade")).over(w0))
  .select("id_usuario","genero","idade","nacionalidade","dias_cargo_anterior","sorriso","oculos")
  .filter(F.col("idade") >20).show(50)
)

+----------+---------+-----+-------------+-------------------+-------+------------+
|id_usuario|   genero|idade|nacionalidade|dias_cargo_anterior|sorriso|      oculos|
+----------+---------+-----+-------------+-------------------+-------+------------+
|     12566|Masculino|   23|     africano|               1186|  10.35|      Nenhum|
|      9275|Masculino|   24|     africano|               2496|  13.02|      Nenhum|
|      4882|Masculino|   25|     africano|                913|  71.17|Oculos_comum|
|       167|Masculino|   27|     africano|               1614|  16.38|      Nenhum|
|     10068| Feminino|   27|     africano|               1157|  85.31|      Nenhum|
|      1548|Masculino|   28|     africano|                182|   94.3|      Nenhum|
|     13717|Masculino|   28|     africano|               1491|  79.92|Oculos_comum|
|      3402|Masculino|   29|     africano|                 92|  84.55|      Nenhum|
|      7393| Feminino|   29|     africano|                792|   97.0|Oculos

#Salvando no bucket

In [ ]:
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')

bucket.blob('saida/linkedin_tratado_pyspark.csv').upload_from_string(df.toPandas().to_csv(index=False), 'text/csv')